In [1]:
import sagas.ofbiz.entities as ee
ent=ee.entity('Product')
val=ent.record('GZ-2002')
print(ent.to_json(val,True))

{'createdStamp': '2019-03-03T22:33:37.435000', 'productName': 'Square Gizmo', 'createdByUserLogin': 'admin', 'productId': 'GZ-2002', 'smallImageUrl': '/images/products/GZ-2002/small.png', 'taxable': 'Y', 'primaryProductCategoryId': '101', 'createdTxStamp': '2019-03-03T22:33:34.232000', 'lastUpdatedTxStamp': '2019-03-03T22:33:54.833000', 'isVirtual': 'N', 'autoCreateKeywords': 'Y', 'description': 'A square gizmo', 'chargeShipping': 'Y', 'internalName': 'Square Gizmo', 'lastModifiedByUserLogin': 'admin', 'lastUpdatedStamp': '2019-03-03T22:33:54.941000', 'lastModifiedDate': '2001-05-13T12:00:00', 'productTypeId': 'FINISHED_GOOD', 'createdDate': '2001-05-13T12:00:00', 'isVariant': 'N', 'largeImageUrl': '/images/products/GZ-2002/large.png'}


In [9]:
from sagas.ofbiz.resources import ResourceDigester

def property_json(prop_key, prop):
    rs={'key':prop_key, 'values':{}}
    for key in prop.values.keys():
        if key not in ['zh-TW']:
            parts=key.split('-')
            rs['values'][parts[0]]=prop.values[key]
    return rs

rd=ResourceDigester(True)
props = rd.get_all_properties()

label='CostComponentType.description.ACTUAL_LABOR_COST'
property_json(label, props[label])

{'key': 'CostComponentType.description.ACTUAL_LABOR_COST',
 'values': {'de': 'Effektive Arbeitskosten',
  'en': 'Actual Labor Cost',
  'es': 'Coste de mano de obra actual',
  'fr': 'Coût réel du travail',
  'it': 'Costo attuale manodopera',
  'ja': '実績労働原価',
  'nl': 'Werk. kosten - Arbeid',
  'ro': 'Cost Actual Manopera',
  'ru': 'Действительные трудозатраты',
  'th': 'ต้นทุนแรงงานโดยแท้จริง',
  'vi': 'Chi phí nhân công thực tế',
  'zh': '实际劳务费用'}}

In [26]:
print(len(props))

14778


In [10]:
label='CostComponentType.description.ACTUAL_LABOR_COST'
property_json(label, props[label])

{'key': 'CostComponentType.description.ACTUAL_LABOR_COST',
 'values': {'de': 'Effektive Arbeitskosten',
  'en': 'Actual Labor Cost',
  'es': 'Coste de mano de obra actual',
  'fr': 'Coût réel du travail',
  'it': 'Costo attuale manodopera',
  'ja': '実績労働原価',
  'nl': 'Werk. kosten - Arbeid',
  'ro': 'Cost Actual Manopera',
  'ru': 'Действительные трудозатраты',
  'th': 'ต้นทุนแรงงานโดยแท้จริง',
  'vi': 'Chi phí nhân công thực tế',
  'zh': '实际劳务费用'}}

In [36]:
props[label].values['de']

'Effektive Arbeitskosten'

In [30]:
props[label].location

'/Users/xiaofeiwu/jcloud/vagrant/fedora/fedora-28/ofbiz/ofbiz-framework/applications/product/config/ProductEntityLabels.xml'

In [39]:
rs=[(k,v.values) for k,v in props.items() if v.location.endswith('ProductEntityLabels.xml')]
print(len(rs))
print(rs[:2])
print('0.de', rs[0][1]['de'])

388
[('ProductContentType.description.XTRA_IMG_1_DETAIL', <google.protobuf.pyext._message.ScalarMapContainer object at 0x121d09e90>), ('FieldDescription.ProductStore.reserveOrderEnumId', <google.protobuf.pyext._message.ScalarMapContainer object at 0x121e940d0>)]
0.de Zusatzbild 1 Detail


In [40]:
labels=[k for k,v in props.items() if v.location.endswith('ProductEntityLabels.xml')]
print(len(labels))

388


In [41]:
def pickups(key, prop, langs):
    rs=[]
    for k,v in prop.values.items():
        if k in langs:
            rs.append({'key':key, 'lang':k, 'value':v})
    return rs

print(type(props[label]))
# props[label]
arranger=[]
arranger.extend(pickups(label, props[label], ['de','fr']))
arranger

<class 'resources_pb2.RsProperty'>


[{'key': 'CostComponentType.description.ACTUAL_LABOR_COST',
  'lang': 'fr',
  'value': 'Coût réel du travail'},
 {'key': 'CostComponentType.description.ACTUAL_LABOR_COST',
  'lang': 'de',
  'value': 'Effektive Arbeitskosten'}]

In [42]:
arranger=[]
for label in labels:
    arranger.extend(pickups(label, props[label], ['de','fr']))
print(len(arranger))

762


In [43]:
class BertManager(object):
    def __init__(self):
        from sagas.conf.conf import cf
        self.ipaddr=cf.conf['bert_servant']
        print(f".. bert service port is {self.ipaddr}")
        self._bc=None
    @property
    def bc(self):
        from bert_serving.client import BertClient
        if self._bc is None:
            self._bc = BertClient(ip=self.ipaddr)
        return self._bc
    def do_test(self):
        """
        $ python -m sagas.train.parallel_corpus do_test
        :return:
        """
        result=self.bc.encode(['First do it', 'then do it right', 'then do it better'])
        print(result)

bm=BertManager()
bm.do_test()

.. bert service port is localhost
[[ 0.27241755  0.05130382  0.28327864 ...  0.7240178   0.15231645
  -0.2559567 ]
 [-0.07413598 -0.22783506 -0.08978284 ...  1.5093101   1.3512012
  -0.03158455]
 [-0.26575348  0.19135515 -0.35613558 ...  1.396218    1.4187485
   0.12651806]]


In [111]:
[row['value'] for row in arranger][:3]

['Zusatzbild 1 Detail',
 'Image - Vue supplémentaire 1 Détaillée',
 'Wählen Sie, wie der Bestand reserviert werden soll.']

In [44]:
doc_vecs=bm.bc.encode([row['value'] for row in arranger])
print(len(doc_vecs), doc_vecs[:2])

762 [[ 0.11969224 -0.30593815  1.4547957  ...  1.6677395   0.8444546
  -0.19611713]
 [-0.41851676 -0.27330032  0.44118643 ...  1.3840292   0.27011162
   0.20589276]]


In [46]:
sample_vecs=doc_vecs[:2]
h, w = sample_vecs.shape
print(h,w)

2 768


In [47]:
h, w = doc_vecs.shape
print(h,w)

762 768


In [48]:
import numpy as np
def save_docs(doc_vecs, file):
    print(len(doc_vecs))
    np.save(file, doc_vecs)  # save

feat_name='samples'
outf="./data/feats/%s.npy"%feat_name
save_docs(doc_vecs, outf)

762


In [115]:
from os import path
print(outf, path.exists(outf))

./data/feats/samples.npy True


In [49]:
import json_utils
meta_outf="./data/feats/%s.json"%feat_name
json_utils.write_json_to_file(meta_outf, arranger)

In [50]:
print(arranger[5])

{'key': 'ProductFeatureType.description.WARRANTY', 'lang': 'fr', 'value': 'Garantie'}


In [107]:
topk=5
def get_similar(questions, doc_vecs, query):
    from sagas.nlu.tts_utils import say_lang
    from termcolor import colored
    
    va = from_redis(r,query)
    if va is None:
        va=bm.bc.encode([query])        
        to_redis(r, va, query)
        
    query_vec = va[0]        
    score = np.sum(query_vec * doc_vecs, axis=1) / np.linalg.norm(doc_vecs, axis=1)
    topk_idx = np.argsort(score)[::-1][:topk]
    print('top %d sentences similar to "%s"' % (topk, colored(query, 'green')))
    for idx in topk_idx:
        row=questions[idx]
        qs=colored(row, 'yellow')
        print('> %s\t%s %s' % (colored('%.1f' % score[idx], 'cyan'), row['key'], qs))

get_similar(arranger, doc_vecs, 'Charge périodique')

h:1, w:768
top 5 sentences similar to "Charge périodique"
> 35.0	ProductPriceType.description.RECURRING_CHARGE {'key': 'ProductPriceType.description.RECURRING_CHARGE', 'lang': 'fr', 'value': 'Charge périodique'}
> 33.3	ShipmentContactMechType.description.SHIP_TO_EMAIL {'key': 'ShipmentContactMechType.description.SHIP_TO_EMAIL', 'lang': 'fr', 'value': "Courriel de l'expédition"}
> 33.2	ProductType.description.DIGITAL_GOOD {'key': 'ProductType.description.DIGITAL_GOOD', 'lang': 'fr', 'value': 'Produit immatériel'}
> 33.1	ProductType.description.FINDIG_GOOD {'key': 'ProductType.description.FINDIG_GOOD', 'lang': 'fr', 'value': 'Produit fini immatériel'}
> 33.0	ProductContentType.description.DETAIL_IMAGE_ALT {'key': 'ProductContentType.description.DETAIL_IMAGE_ALT', 'lang': 'fr', 'value': "Texte alternatif de l'image - Détail"}


In [109]:
get_similar(arranger, doc_vecs, 'périodique')

h:1, w:768
top 5 sentences similar to "périodique"
> 32.1	ProductPriceType.description.RECURRING_CHARGE {'key': 'ProductPriceType.description.RECURRING_CHARGE', 'lang': 'fr', 'value': 'Charge périodique'}
> 31.9	ProductFeatureType.description.TOPIC {'key': 'ProductFeatureType.description.TOPIC', 'lang': 'fr', 'value': 'Matière'}
> 31.7	ProductCategoryType.description.MATERIALS_CATEGORY {'key': 'ProductCategoryType.description.MATERIALS_CATEGORY', 'lang': 'fr', 'value': 'Matériel'}
> 31.7	GoodIdentificationType.description.OTHER_ID {'key': 'GoodIdentificationType.description.OTHER_ID', 'lang': 'fr', 'value': 'Autre'}
> 31.7	SubscriptionType.description.OTHER_SUBSCR {'key': 'SubscriptionType.description.OTHER_SUBSCR', 'lang': 'fr', 'value': 'Autre'}


In [66]:
query='Charge périodique'
query_vec = bm.bc.encode([query])
h, w = query_vec.shape
print(h,w)
# query_vec

1 768


In [95]:
import struct
import redis
import numpy as np

def toRedis(r,a,n):
   """Store given Numpy array 'a' in Redis under key 'n'"""
   h, w = a.shape
   shape = struct.pack('>II',h,w)
   encoded = shape + a.tobytes()

   # Store encoded data in Redis
   r.set(n,encoded)
   return

def fromRedis(r,n):
   """Retrieve Numpy array from Redis key 'n'"""
   encoded = r.get(n)
   h, w = struct.unpack('>II',encoded[:8])
   a = np.frombuffer(encoded, dtype=np.uint16, offset=8).reshape(h,w)
   return a

# Create 80x80 numpy array to store
a0 = np.arange(6400,dtype=np.uint16).reshape(80,80) 
print(a0.shape, a0.dtype)
# Redis connection
r = redis.Redis(host='localhost', port=6379, db=0)

# Store array a0 in Redis under name 'a0array'
toRedis(r,a0,'a0array')

# Retrieve from Redis
a1 = fromRedis(r,'a0array')

np.testing.assert_array_equal(a0,a1)

(80, 80) uint16


In [86]:
print(query_vec.shape, query_vec.dtype)

(1, 768) float32


In [87]:
toRedis(r,query_vec,query)

In [96]:
def to_redis(r,a,n):
   """Store given Numpy array 'a' in Redis under key 'n'"""
   h, w = a.shape
   shape = struct.pack('>II',h,w)
   encoded = shape + a.tobytes()

   # Store encoded data in Redis
   r.set(n,encoded)
   return

def from_redis(r,n):
    """Retrieve Numpy array from Redis key 'n'"""
    encoded = r.get(n)
    if encoded is None:
        return None
    h, w = struct.unpack('>II',encoded[:8])
    print(f"h:{h}, w:{w}")
    a = np.frombuffer(encoded, dtype=np.float32, offset=8).reshape(h,w)
    return a

# Retrieve from Redis
a1 = from_redis(r,query)
# print(a1)
h, w = a1.shape
print(h,w)

h:1, w:768
1 768


In [88]:
ae=r.get('unexists')
ae is None

True

In [113]:
from sagas.nlu.inspector_fixtures import InspectorFixture
from sagas.nlu.inspector_common import Context
def expand(data, keys, domains):
    fixt=InspectorFixture()
    domains, meta=fixt.request_domains(data)
    ctx=Context(meta, domains)
    # execute get_similar with ctx.chunks
    
texts = ['Shenzhen ist das Silicon Valley für Hardware-Firmen',
                 'Ich stimme dir in diesem Punkt nicht zu.',
                 'Die Nutzung der Seite ist kostenlos.',
                 ]
for text in texts:
    data = {'lang': 'de', "sents": text}
    expand(data)
    print('✁', '-'*30)

.. request is {'lang': 'de', 'sents': 'Shenzhen ist das Silicon Valley für Hardware-Firmen'}


AttributeError: 'TransClipConf' object has no attribute 'common_s'